In [1]:
import os
import time
from tqdm import tqdm, trange
os.environ['CUDA_LAUNCH_BLOCKING'] = '1' 
class Config:
    def __init__(self, ent_num, rel_num):
        self.dim =  200
        self.ent_num = ent_num
        self.rel_num = rel_num
        self.inputDrop = 0.2
        self.hideDrop = 0.3
        self.featureDrop = 0.3
        self.filter_num = 64
        self.dim1 = 20
        self.hide_size = 19456
        self.smooth = 0.1
        self.lr = 0.0001
        self.weight_decay = 0.0
        self.epochs = 300
        self.batch_size = 128

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import  Parameter
class ConvE(nn.Module):
    def __init__(self, config):
        super(ConvE, self).__init__()
        self.config = config
        self.ent_embs = nn.Embedding(self.config.ent_num, self.config.dim)
        self.rel_embs = nn.Embedding(self.config.rel_num, self.config.dim)
        self.input_drop = nn.Dropout(config.inputDrop)
        self.hide_drop = nn.Dropout(config.hideDrop)
        self.feature_drop = nn.Dropout2d(config.featureDrop)
        self.conv = nn.Conv2d(1, 64, (3, 3), bias=True)
        self.bn0 = nn.BatchNorm2d(1)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm1d(config.dim)
        self.fc = nn.Linear(config.hide_size, config.dim)
        self.dim = config.dim #dim = 200
        self.dim1 = config.dim1  #dim1 = 20
        self.dim2 = self.dim // self.dim1 # dim2 = 10
        self.loss = nn.BCELoss()
        self.register_parameter('b',Parameter(torch.zeros(config.ent_num)))
        self.init()
        
    def init(self):
        nn.init.xavier_normal_(self.ent_embs.weight.data)
        nn.init.xavier_normal_(self.rel_embs.weight.data)
    def forward(self, e1, rel):
        e1_emb = self.ent_embs(e1).view(-1, 1, self.dim1, self.dim2)#el_emb; batch*1*20*10
        rel_emb = self.rel_embs(rel).view(-1, 1 ,self.dim1, self.dim2)
        conv_input = torch.cat([e1_emb, rel_emb], dim = 2)#con_input: bath*1*40*10
        conv_input = torch.transpose(conv_input, 2, 1).reshape((-1, 1, 2*self.dim1, self.dim2))
        conv_input = self.bn0(conv_input)
        x = self.input_drop(conv_input)
        x = self.conv(conv_input)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.feature_drop(x)
        x = x.view(x.shape[0], -1)#bacth*hide_size(38*8*32 = 9728)
        x = self.fc(x)
        x = self.hide_drop(x)
        x = self.bn2(x)
        x = F.relu(x)#batch*dim          ent_ems.weight   dim*ent_num
        #print(x.shape, self.ent_embs.weight.shape)
        x = torch.mm(x, self.ent_embs.weight.transpose(1, 0))
        x += self.b.expand_as(x)
        pred = torch.sigmoid(x)
        return pred
        
        
        
        

In [3]:
#config = Config(4, 4)
#model = ConvE(config)
#e1 = torch.LongTensor([1, 2, 1])
#rel = torch.LongTensor([1, 2, 3])
#print(model(e1, rel))

In [4]:
from collections import defaultdict as ddict
class dataProcess:
    def __init__(self, file_name):
        self.dir = '/home/qiupp/code/ConvE/data/'+file_name+'/'
        self.ent2id = {}
        self.rel2id = {}
        self.data = {sql:self.read(sql) for sql in ['train', 'valid', 'test']}
        self.train_sr2o = {}
        self.all_sr2o = ddict(set)
        self.triples = ddict(list)
        self.sub_rel_map_obj()
        self.train_triples()
        self.valid_or_test_triples()
        
    def read(self, file_type):
        with open(self.dir+file_type+'.txt', 'r') as f:
            lines = f.readlines()
        triples = []
        for line in lines:
            temp = line.strip().split()
            triples.append((self.get_ent_id(temp[0]), self.get_rel_id(temp[1]), self.get_ent_id(temp[2])))
        return triples
    def get_ent_id(self, ent):
        if  not ent in self.ent2id.keys():
            self.ent2id[ent] = len(self.ent2id)
        return self.ent2id[ent]
        
    def get_rel_id(self, rel):
        if not rel  in self.rel2id.keys():
            self.rel2id[rel] = len(self.rel2id)
        return self.rel2id[rel]
    def ent_num(self):
        return len(self.ent2id)
    def rel_num(self):
        return len(self.rel2id)
    def sub_rel_map_obj(self):
        for fact in self.data['train']:
            h, r, t = fact
            self.all_sr2o[(h, r)].add(t)
            self.all_sr2o[(t, r+ self.rel_num())].add(h)
        
        self.train_sr2o = {k: list(v) for k, v in self.all_sr2o.items()}
        for sql in ['valid', 'test']:
            for fact in self.data[sql]:
                h, r, t = fact
                self.all_sr2o[(h, r)].add(t)
                self.all_sr2o[(t,r+self.rel_num())].add(h)
    def train_triples(self):
        for (h, r), t in self.train_sr2o.items():
            self.triples['train'].append({'triple':(h, r), 'labels':t})
    def valid_or_test_triples(self):
        for sql in ['valid', 'test']:
            for h, r, t in self.data[sql]:
                rev_rel = r + self.rel_num()
                self.triples[sql+'_tail'].append({'triple':(h, r, t), 'labels':self.all_sr2o[(h, r)]})
                self.triples[sql+'_head'].append({'triple':(t,rev_rel, h), 'labels':self.all_sr2o[(t, rev_rel)]})
        

In [5]:
from torch.utils.data import DataLoader, Dataset
class MyDataSet(Dataset):
    def __init__(self, dataPro, type, smooth):
        self.type = type
        self.data = dataPro.triples[type]
        self.ent_num = dataPro.ent_num()
        self.smooth = smooth
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        element = self.data[index]
        
        triple = torch.LongTensor(element['triple'])
        labels = self.get_label(element['labels'])
        return triple, labels
    def get_label(self, labels):
        temp_label = torch.zeros(self.ent_num)
        for label in labels:
            temp_label[label] = 1
        if self.smooth != 0.0:
            temp_label = (1-self.smooth)*temp_label+1.0/self.ent_num
        return torch.FloatTensor(temp_label)
        
        

In [6]:
class Measure:
    def __init__(self):
        self.hit1 = 0.0
        self.hit3 = 0.0
        self.hit10 = 0.0
        self.mr = 0.0
        self.mrr = 0.0
    def updata(self, rank):
        if rank == 1:
            self.hit1 += 1.0
        if rank <= 3:
            self.hit3 += 1.0
        if rank <= 10:
            self.hit10 += 1.0
        self.mr += rank
        self.mrr += 1.0 / rank
    def deal(self, facts_num):
        self.hit1 /=  facts_num
        self.hit3 /= facts_num
        self.hit10 /= facts_num
        self.mr /= facts_num
        self.mrr /= facts_num
    def print_(self):
        print('hit1: '+str(self.hit1))
        print('hit3: '+str(self.hit3))
        print('hit10: '+str(self.hit10))
        print('mr: '+str(self.mr))
        print('mrr: '+str(self.mrr))
    def init(self):
        self.hit1 = 0.0
        self.hit3 = 0.0
        self.hit10 = 0.0
        self.mr = 0.0
        self.mrr = 0.0
    
        
        

In [7]:
import numpy as np
import os
class Trainer:
    def __init__(self, dataPro, config, device):
        self.device = device
        self.dataPro = dataPro
        self.mydataset = MyDataSet(dataPro, 'train', 0.1)
        self.measure = Measure()
        self.model = ConvE(config).to(device)
        self.config = config
        self.valid_data = DataLoader(MyDataSet(dataPro, 'valid_tail',0.00), batch_size = config.batch_size)
        self.valid_data_len = len(dataPro.triples['valid_tail'])
        print("valid_data_len"+str(self.valid_data_len))
        
    def MyTrain(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.lr,weight_decay=self.config.weight_decay )
        dataload = DataLoader(self.mydataset, batch_size = self.config.batch_size, shuffle = True)
        for epoch in trange(1,self.config.epochs+1):
            self.model.train()
            total_loss = 0.0
            for i,(triples, labels) in enumerate(dataload):
                triples = triples.to(self.device)
                labels = labels.to(self.device)
                e1 = triples[:, 0]
                rel = triples[:, 1]
                optimizer.zero_grad()
                #print(e1, rel)
                score = self.model(e1, rel)
#                 if i% 500 == 0:
#                     print(score[1])
                loss = self.model.loss(score, labels)
                total_loss += loss.cpu().item()
                loss.backward()
                optimizer.step()
            #total_loss_to_tensor = torch.FloatTensor(total_loss)
            #print(total_loss_to_tensor)
            print("损失值为："+str(total_loss))
            
            if epoch % 1 == 0:
                self.save_model(epoch)
                self.measure.init()
                self.model.eval()
                with torch.no_grad():
                    tot_num = 0
                    for i, (triples, labels) in enumerate(self.valid_data):
                        tot_num += e1.shape[0]
                        e1 = triples[:, 0]
                        rel = triples[:, 1]
                        e2 = triples[:, 2]
                        e1 = e1.to(self.device)
                        rel = rel.to(self.device)
                        pred = self.model(e1, rel)
                        pred = pred.cpu()
                        self.predict(e2, labels,pred)
                    self.measure.deal(tot_num)
                    #print(tot_num)
                    self.measure.print_()
                 
                
    
    def save_model(self, epoch):
        model_path = '/home/qiupp/code/ConvE/models/'
        if not os.path.exists(model_path):
            os.mkdir(model_path)
        torch.save(self.model, model_path+str(epoch)+'.pkl')
    def predict(self, e2, labels, pred):
        
        n_batch = torch.arange(e2.shape[0])
        target = pred[n_batch, e2]
        pred = torch.where(labels.byte(), torch.zeros_like(pred), pred)
        pred[n_batch, e2] = target
        #print(target)
        pred = pred.detach().numpy()
        e2 = e2.numpy()
        for i in range(e2.shape[0]):
            ele = e2[i]
            one_pred =  pred[i]
            sc = one_pred[ele]
            one_pred = np.delete(one_pred, ele)
            one_pred = np.insert(one_pred,0, sc )
            rank = (one_pred[0] <= one_pred).sum()
            #print(rank)
            self.measure.updata(rank)
        
        
        

In [8]:
dataPro = dataProcess('FB15k-237')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
config = Config(dataPro.ent_num(), dataPro.rel_num()*2)
print(len(dataPro.data['valid']))
trainer = Trainer(dataPro,config, device)
trainer.MyTrain()



17535
valid_data_len17535


  0%|                                                                                                   | 0/300 [00:00<?, ?it/s]

损失值为：123.67273036297411


/tmp/ipykernel_2648734/61736070.py:70: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:328.)
  pred = torch.where(labels.byte(), torch.zeros_like(pred), pred)
  0%|▎                                                                                        | 1/300 [00:24<2:00:25, 24.16s/it]

hit1: 0.09544803893501289
hit3: 0.10775837389063841
hit10: 0.14285714285714285
mr: 4815.245805897509
mrr: 0.11077693791927085
损失值为：9.453660823404789


...

hit1: 0.33381047809905523
hit3: 0.4718007443458345
hit10: 0.6177497852848555
mr: 160.71102204408817
mrr: 0.4284874640260654
损失值为：1.4081622103694826


 39%|█████████████████████████████████▍                                                   | 118/300 [1:29:43<2:32:06, 50.14s/it]

hit1: 0.33529916976810764
hit3: 0.4719152590896078
hit10: 0.6180933295161752
mr: 159.99421700543945
mrr: 0.429016477505457


 39%|█████████████████████████████████▍                                                   | 118/300 [1:30:06<2:18:59, 45.82s/it]


KeyboardInterrupt: 

In [9]:
class Tester:
    def __init__(self,dataPro, model_path, device):
        self.model = torch.load(model_path, map_location = device)
        self.test_tail = MyDataSet(dataPro, 'test_tail', 0.0)
        self.test_head = MyDataSet(dataPro, 'test_head', 0.0)
        self.lens = len(dataPro.data['test'])
        self.test_tail_load = DataLoader(self.test_tail, batch_size = 128)
        self.test_head_load = DataLoader(self.test_head, batch_size = 128)
        self.measure = Measure()
        self.device = device
        self.model.eval()
    def test(self):
        with torch.no_grad():
            self.measure.init()
            for i, (triples, labels) in enumerate(self.test_tail_load):
                e1 = triples[:, 0]
                rel = triples[:,1]
                if i == 0:
                    print(labels)
                #print(e1)
                e1 = e1.to(self.device)
                rel = rel.to(self.device)
                pred = self.model(e1, rel)
                #print(pred)
                pred = pred.cpu()
                e2 = triples[:, 2]
                self.predict(e2, labels, pred)
            print(self.measure.mrr)
            self.measure.deal(self.lens)
            self.measure.print_()
            self.measure.init()
        
            for i, (triples, labels) in enumerate(self.test_head_load):
            
                e1 = triples[:, 0]
                rel = triples[:,1]
                e1 = e1.to(self.device)
                rel = rel.to(self.device)
                pred = self.model(e1, rel)
                pred = pred.cpu()
                e2 = triples[:, 2]
                self.predict(e2, labels, pred)
            self.measure.deal(self.lens)
            self.measure.print_()
        
            
            
    def predict(self, e2, labels, pred):
        
        n_batch = torch.arange(e2.shape[0])
        target = pred[n_batch, e2]
        pred = torch.where(labels.byte(), torch.zeros_like(pred), pred)
        pred[n_batch, e2] = target
        #print(target)
        pred = pred.detach().numpy()
        e2 = e2.numpy()
        for i in range(e2.shape[0]):
            ele = e2[i]
            one_pred =  pred[i]
            sc = one_pred[ele]
            one_pred = np.delete(one_pred, ele)
            one_pred = np.insert(one_pred,0, sc )
            rank = (one_pred[0] <= one_pred).sum()
            #print(rank)
            self.measure.updata(rank)    
            
    
    
    
  

In [10]:
model_path =  "/home/qiupp/code/ConvE/models/118.pkl"
tester = Tester(dataPro, model_path, device)
tester.test()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
8541.983142886143
hit1: 0.32018958272256426
hit3: 0.46100850190559955
hit10: 0.6100361575295612
mr: 176.86157529561223
mrr: 0.4173743351356466
hit1: 0.13402716700869735
hit3: 0.23624548030880485
hit10: 0.379849506498583
mr: 344.93643115410924
mrr: 0.21511006519040557


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


KeyError: 'a'

0.9688639316269664


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [1]:

import torch
torch.randn((1, 2))

tensor([[-1.7090,  1.3172]])